# 创建 MCP 服务

AgentKit 网关支持自有和公共 MCP 服务的快速部署，并且支持将 Local MCP 转换为 Remote MCP 并提供远程访问的能力。部署自有 MCP 服务时，AgentKit 网关可以访问你的函数服务中的 MCP 服务，也可以通过公共包或你的镜像仓库中的私有镜像再函数服务中创建 MCP 服务。部署公共 MCP 服务时，AgentKit 网关可以通过固定域名的形式访问外部的 MCP 服务，支持 API Key 形式的出站认证。无论是部署自有 MCP 服务还是公共 MCP 服务，AgentKit 网关都支持 API Key 和 OAuth2 JWT 形式的入站认证，为你的服务安全保驾护航。

在本教程中，我们将演示如何通过公共包和固定域名部署自有 MCP 服务并为服务增加 API Key 入站认证和 API Key 出站认证，你将学会，

1. 认识 AgentKit 网关，
2. 如何在 AgentKit 网关中部署 MCP 服务，
3. 如何在 AgentKit 网关中为 MCP 服务配置入站认证。
4. 如何在 AgentKit 网关中为 MCP 服务配置出站认证。

在开始本教程之前，请确保你已经**正确配置了凭证**。AgentKit 需要两类凭证：

1. SDK 与 CLI 的云服务访问凭证

   云服务访问凭证用于创建和管理云资源，你可以通过 `agentkit` 命令行工具全局配置。

   ```sh
   agentkit config --global --set volcengine.access_key=YOUR_ACCESS_KEY
   agentkit config --global --set volcengine.secret_key=YOUR_SECRET_KEY
   agentkit config --global --set volcengine.region=cn-beijing
   ```

2. Agent 运行时的环境变量

   环境变量用于 Agent 访问大模型等服务。你可以在 `agentkit.yaml` 的 `runtime_envs` 中配置。

   ```python
   config.add_runtime_env("MODEL_AGENT_NAME", "your-model-name")
   config.add_runtime_env("MODEL_AGENT_API_KEY", "your-model-api-key")
   ```

> **注意**：如果你还没有配置凭证，请先学习 [`01-tutorials/01-runtime/00_configuration_and_credentials.ipynb`](../01-runtime/00_configuration_and_credentials.ipynb) 教程完成配置。

## 1. 为公共包创建 MCP 服务

AgentKit 网关支持通过公共包或镜像仓库中的私有镜像创建 MCP 服务，也支持通过函数服务或固定域名创建内部或外部的 MCP 服务。在创建 MCP 服务时，AgentKit 网关将要求您选择通过 API Key 或 OAuth2 JWT进行入站认证。入站认证可以保证只有您信任的对象才能访问您的 MCP 服务。当请求抵达 AgentKit 网关时，对于 API Key 类型的入站认证，AgentKit 网关将校验 Authorization 请求头中的 API Key 是否和预先创建的 API Key 一致，对于 OAuth2 JWT 类型的入站认证，AgentKit 网关将校验 Authorization 请求头中的 JWT Token 的颁发者（issuer）、客户端 ID、权限范围（scopes）和受众（audiences）是否和你预先设置的认证策略一致。更多关于入站认证的细节将在后续的教程中介绍。

首先，我们将为公共包创建 MCP 服务。公共包是单个 MCP 服务器配置信息。例如

```json
{
  "mcpServers": {
    "sequential-thinking": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-sequential-thinking"
      ]
    }
  }
}
```

是 `@modelcontextprotocol/server-sequential-thinking` MCP 服务器的公共包。通过公共包创建 MCP 服务时，AgentKit 网关将在函数服务中按照 Local MCP 的方式部署 MCP 服务器，然后将它转换为 Remote MCP 并通过 AgentKit 网关提供远程访问的能力。

接下来，我们将为 `@modelcontextprotocol/server-sequential-thinking` 这一公共包部署 MCP 服务。

In [ ]:
from agentkit.sdk.mcp.client import AgentkitMCPClient
from agentkit.sdk.mcp.types import (
    CreateMCPServiceRequest,
    NetworkForCreateMCPService,
    InboundAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService,
    BackendForCreateMCPService,
    BackendCustomMcpForCreateMCPService,
    BackendCustomMcpPublicPackageForCreateMCPService,
)
import json
import uuid

id = str(uuid.uuid4())[-8:]
mcp_config = {
    "mcpServers": {
        "sequential-thinking": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-sequential-thinking"],
        },
    },
}

client = AgentkitMCPClient()
request = CreateMCPServiceRequest(
    Name=f"mcp-service-{id}",
    ProtocolType="MCP",
    # 通过 `/mcp` 路径，访问 AgentKit 网关 MCP 服务。
    Path="/mcp",
    # 通过公网访问 MCP 服务。
    NetworkConfiguration=NetworkForCreateMCPService(
        EnablePublicNetwork=True,
        EnablePrivateNetwork=False,
    ),
    # 使用 API Key 入站认证保证服务安全。
    InboundAuthorizerConfiguration=InboundAuthorizerForCreateMCPService(
        AuthorizerType="ApiKey",
        Authorizer=InboundAuthorizerAuthorizerForCreateMCPService(
            KeyAuth=InboundAuthorizerAuthorizerKeyAuthForCreateMCPService(
                ApiKeys=[
                    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name=f"mcp-service-api-key-{id}",
                    ),
                ],
            ),
        ),
    ),
    # AgentKit 网关 MCP 服务指向公共包。
    BackendType="CustomPublic",
    BackendConfiguration=BackendForCreateMCPService(
        CustomMcpConfiguration=BackendCustomMcpForCreateMCPService(
            PublicPackage=BackendCustomMcpPublicPackageForCreateMCPService(
                McpType="Remote",
                PackageManagerType="NPX",
                RawConfig=json.dumps(mcp_config),
            )
        ),
    ),
    ProjectName="default",
)

mcp_service_id = ""
try:
    response = client.create_mcp_service(request)
    mcp_service_id = response.mcp_service_id
    print(f"MCP service created successfully! MCP service ID: {mcp_service_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

AgentKit Gateway 部署 MCP 服务需要一段时间。在等待服务部署的时间里，你可以泡一杯咖啡歇息一会儿。等到 MCP 服务部署完成后，我们测试一下刚刚部署的 `@modelcontextprotocol/server-sequential-thinking` MCP 服务。

In [ ]:
import time
from agentkit.sdk.mcp.types import (
    GetMCPServiceRequest,
)

request = GetMCPServiceRequest(
    MCPServiceId=mcp_service_id,
)

endpoint = ""
api_key = ""
try:
    while True:
        response = client.get_mcp_service(request)
        if response.mcp_service.status == "Ready":
            endpoint = response.mcp_service.network_configurations[0].endpoint
            api_key = response.mcp_service.inbound_authorizer_configuration.authorizer.key_auth.api_keys[0].key
            break
        if response.mcp_service.status != "Creating":
            print(f"The MCP service is in {response.mcp_service.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

url = f"{endpoint}/mcp"
print(f"MCP service URL: {url}")
print(
    f'MCP service API key: {api_key}\n  (Hint: Add a "Authorization: Bearer {api_key}" request header to access the MCP service)'
)

In [ ]:
!pip install fastmcp --quiet

In [ ]:
from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport

transport = StreamableHttpTransport(
    url=url,
    headers={
        "Authorization": f"Bearer {api_key}",
    }
)
mcp_client = Client(transport)

async with mcp_client:
    tools = await mcp_client.list_tools()
    print(f"The MCP server has a \"{tools[0].name}\" MCP tool.")

恭喜！你已经使用 AgentKit 网关部署了第 1 个 MCP 服务，这个 MCP 服务包含 1 个名为 `sequentialthinking` 的 MCP 工具。接下来，我们将尝试通过固定域名部署 MCP 服务。

## 2. 为固定域名创建 MCP 服务

公共 MCP 服务大多通过固定域名的形式访问，例如高德地图 MCP 服务域名是 `mcp.amap.com`，GitHub MCP 服务域名是 `api.githubcopilot.com/mcp`。大多数公共 MCP 服务都需要携带 API Key 访问。如果你希望通过固定域名访问内部的 MCP 服务，或是二次分发公共 MCP 服务，你可以使用 AgentKit 网关为固定域名创建 MCP 服务。AgentKit 网关 MCP 服务支持 API Key 类型的出站认证，也就是说，AgentKit 网关将为出网请求自动带上公共 MCP 服务所需的 API Key。更多关于出站认证的细节将在后续的教程中介绍。

我们刚刚通过公共包部署了 `@modelcontextprotocol/server-sequential-thinking` 的 MCP 服务。并且获得了 MCP 服务的访问地址和 API Key。接下来，我们将为这一个 URL，通过固定域名的形式，再次创建 MCP 服务。

In [ ]:
from urllib.parse import urlparse
from agentkit.sdk.mcp.types import (
    BackendCustomForCreateMCPService,
    OutboundAuthorizerForCreateMCPService,
    OutboundAuthorizerAuthorizerForCreateMCPService,
    OutboundAuthorizerAuthorizerKeyAuthForCreateMCPService,
    OutboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService,
)

parsed_url = urlparse(url)
domain = parsed_url.netloc

id_2 = str(uuid.uuid4())[-8:]

client = AgentkitMCPClient()
request = CreateMCPServiceRequest(
    Name=f"mcp-service-{id_2}",
    ProtocolType="MCP",
    # 通过 `/mcp` 路径，访问 AgentKit 网关 MCP 服务。
    Path="/mcp",
    # 通过公网访问 MCP 服务。
    NetworkConfiguration=NetworkForCreateMCPService(
        EnablePublicNetwork=True,
        EnablePrivateNetwork=False,
    ),
    # 使用 API Key 入站认证保证服务安全。
    InboundAuthorizerConfiguration=InboundAuthorizerForCreateMCPService(
        AuthorizerType="ApiKey",
        Authorizer=InboundAuthorizerAuthorizerForCreateMCPService(
            KeyAuth=InboundAuthorizerAuthorizerKeyAuthForCreateMCPService(
                ApiKeys=[
                    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name=f"mcp-service-api-key-{id_2}",
                    ),
                ],
            ),
        ),
    ),
    # AgentKit 网关 MCP 服务指向固定域名。
    BackendType="Domain",
    BackendConfiguration=BackendForCreateMCPService(
        CustomConfiguration=BackendCustomForCreateMCPService(
            Domain=domain,
            ProtocolType="HTTP",
            Port=80,
        )
    ),
    # 使用 API Key 出站认证为出站请求添加凭据。
    OutboundAuthorizerConfiguration=OutboundAuthorizerForCreateMCPService(
        AuthorizerType="ApiKey",
        Authorizer=OutboundAuthorizerAuthorizerForCreateMCPService(
            KeyAuth=OutboundAuthorizerAuthorizerKeyAuthForCreateMCPService(
                ApiKeyLocation="Header",
                Parameter="Authorization",
                ApiKeys=[
                    OutboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name="",
                        Key=f"Bearer {api_key}",
                    ),
                ],
            ),
        ),
    ),
    ProjectName="default",
)

mcp_service_id_2 = ""
try:
    response = client.create_mcp_service(request)
    mcp_service_id_2 = response.mcp_service_id
    print(f"MCP service created successfully! MCP service ID: {mcp_service_id_2}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

In [ ]:
request = GetMCPServiceRequest(
    MCPServiceId=mcp_service_id_2,
)

endpoint_2 = ""
api_key_2 = ""
try:
    while True:
        response = client.get_mcp_service(request)
        if response.mcp_service.status == "Ready":
            endpoint_2 = response.mcp_service.network_configurations[0].endpoint
            api_key_2 = response.mcp_service.inbound_authorizer_configuration.authorizer.key_auth.api_keys[0].key
            break
        if response.mcp_service.status != "Creating":
            print(f"The MCP service is in {response.mcp_service.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

url_2 = f"{endpoint_2}/mcp"
print(f"MCP service URL: {url_2}")
print(
    f'MCP service API key: {api_key_2}\n  (Hint: Add a "Authorization: Bearer {api_key_2}" request header to access the MCP service)'
)

In [ ]:
from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport

transport = StreamableHttpTransport(
    url=url_2,
    headers={
        "Authorization": f"Bearer {api_key_2}",
    }
)
mcp_client = Client(transport)

async with mcp_client:
    tools = await mcp_client.list_tools()
    print(f"The MCP server has a \"{tools[0].name}\" MCP tool.")

恭喜！你已经使用 AgentKit 网关部署了第 2 个 MCP 服务，这个 MCP 服务通过固定域名访问了 `@modelcontextprotocol/server-sequential-thinking` MCP 服务。

## 总结

恭喜你！现在你已经掌握了创建 AgentKit 网关 MCP 服务的方法，

1. 通过公共包创建 MCP 服务。
2. 通过固定域名创建 MCP 服务。

现在，请继续学习其他教程。